**Disclaimer**: This feature hits the LLMs limits really hard, so don't expect it to output better graphs than if you where programming it yourself

In [1]:
import dspy
from hybridagi.core.pipeline import Pipeline
from hybridagi.core.datatypes import AgentState, Document
from hybridagi.modules.agents.tools import PredictTool, SpeakTool, AskUserTool
from hybridagi.modules.extractors import PlanExtractor, GraphProgramExtractor

lm = dspy.OllamaLocal(model='mistral', max_tokens=1024, stop=["\n\n\n", "\n---"])
dspy.configure(lm=lm)

agent_state = AgentState()

tools = [
    PredictTool(),
    SpeakTool(
        agent_state = agent_state,
    ),
    AskUserTool(
        agent_state = agent_state,
    ),
]

program_description = \
"""
Write a poem about AI
"""

input_doc = Document(text=program_description)

pipeline = Pipeline()

pipeline.add(
    PlanExtractor(
        tools=tools,
    ),
    GraphProgramExtractor(
        tools=tools,
    )
)

graph_programs = pipeline(input_doc)

output_graph = graph_programs.progs[0]

print(output_graph)

output_graph.show(notebook=True)

/home/yoan/.cache/pypoetry/virtualenvs/hybridagi-B1GoJrSC-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 1/1 [00:14<00:00, 14.37s/it]

// @desc: A graph for generating and speaking a poem based on user input.
CREATE
// Nodes declaration
(start:Control {id: "start"}),
(end:Control {id: "end"}),
(ask_question:Action {
  id: "ask_question",
  purpose: "Ask user for a topic",
  tool: "AskUser",
  prompt: "What topic would you like me to write a poem about?"
}),
(generate_poem:Action {
  id: "generate_poem",
  purpose: "Generate a poem based on the given topic",
  tool: "Predict",
  prompt: "Write a poem about AI"
}),
(speak_poem:Action {
  id: "speak_poem",
  purpose: "Speak the generated poem to the user",
  tool: "Speak"
}),
(is_answer_valid:Decision {
  id: "is_answer_valid",
  purpose: "Check if the given topic is valid",
  question: "Is the given topic valid?"
}),
// Structure declaration
(start)-[:NEXT]->(ask_question),
(ask_question)-[:NEXT]->(is_answer_valid),
(generate_poem)-[:NEXT]->(speak_poem),
(speak_poem)-[:NEXT]->(end),
(is_answer_valid)-[:YES]->(generate_poem),
(is_answer_valid)-[:NO]->(end)
